# Covid-19 Testing Importance

## Introduction

I believe that testing is one of the most crucial parts of dealing with an epidemic virus. Testing helps us identify and isolate positive cases. The more tests you perform, the faster you isolate the case preventing them from coming into contact with others, **slowing the rate of transmission**.  
  
**Date this notebook was written: 19/3/2020**  
  
**Disclaimer**: *In any way I do not want to point my finger on the governments and people of countries. It is not possible to know what were the reasons and the circumstances that lead to a lack of testing. My only target is to see if the data suggests that testing has a major role on this specific epidemic.*

In [21]:
import requests
import re

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup

## Data sources

The first thing we want to find is how many tests have been performed on each country. The data will be extracted from this page: https://ourworldindata.org/coronavirus-testing-source-data . You may observe that the dates of last report are not the same for each country. This will be taken into account.

In [41]:
response = requests.get('https://ourworldindata.org/coronavirus-testing-source-data')
soup = BeautifulSoup(response.content)

table_soup = soup.find("div", {"class": "tableContainer"}).findAll("tr")[1:] # skip the headers

# RegEx to extract the country name, number of test, date
reg = re.compile("<tr><td>(.+)<\/td><td>([\d,]+)<\/td><td>([\w\s]+)<\/td><td>.*<\/td><td>.*<\/td><\/tr>")
testing_data = [reg.match(str(row)).groups() for row in table_soup]

# Crete the dataframe
testing_df = pd.DataFrame(testing_data, columns=["Country_Province", "Tests", "Date"])
testing_df['Tests'] = testing_df['Tests'].str.replace(',','').astype(int)    # Transform the x,xxx string to integers
testing_df['Date'] = pd.to_datetime(testing_df['Date'])    # Cast Date from string to date type
display(testing_df)

,Country_Province,Tests,Date
0,Armenia,694,2020-03-16
1,Australia – Government of the Australian Capit...,1391,2020-03-17
2,Australia – New South Wales,30244,2020-03-17
3,Austria,10278,2020-03-17
4,Bahrain,13553,2020-03-17
...,...,...,...
59,United Arab Emirates,125000,2020-03-16
60,United Kingdom,50442,2020-03-17
61,United States – CDC samples tested,21105,2020-03-11
62,United States – COVID-Tracking project,41552,2020-03-16
